In [2]:
from openai import OpenAI
import requests
import openai
from dotenv import load_dotenv
import os
load_dotenv()

client = OpenAI()

result = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is the weather in Tokyo?",
        }
    ],
    model="gpt-4o",
)

print(result.choices[0].message.content)

I'm sorry, but I can't provide real-time weather updates. For the current weather in Tokyo, please check a reliable weather website or use a weather app.


In [3]:
def get_weather(city_name):
    # Replace 'YOUR_API_KEY' with your actual WeatherAPI key
    api_key = os.getenv("WEATHER_API_KEY")
    base_url = "http://api.weatherapi.com/v1/current.json"

    # Parameters for the API request
    params = {
        "key": api_key,
        "q": city_name,  # City name (e.g., "Paris")
    }

    try:
        # Make the GET request
        response = requests.get(base_url, params=params)

        # Check if request was successful
        if response.status_code == 200:
            # Parse JSON response
            data = response.json()
            return data

        else:
            print(
                f"The Weather API cannot respond due to an error")

    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
get_weather('Tokyo')

{'location': {'name': 'Tokyo',
  'region': 'Tokyo',
  'country': 'Japan',
  'lat': 35.6895,
  'lon': 139.6917,
  'tz_id': 'Asia/Tokyo',
  'localtime_epoch': 1742326093,
  'localtime': '2025-03-19 04:28'},
 'current': {'last_updated_epoch': 1742325300,
  'last_updated': '2025-03-19 04:15',
  'temp_c': 4.4,
  'temp_f': 39.9,
  'is_day': 0,
  'condition': {'text': 'Light rain',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/296.png',
   'code': 1183},
  'wind_mph': 11.0,
  'wind_kph': 17.6,
  'wind_degree': 351,
  'wind_dir': 'N',
  'pressure_mb': 1005.0,
  'pressure_in': 29.68,
  'precip_mm': 3.93,
  'precip_in': 0.15,
  'humidity': 90,
  'cloud': 100,
  'feelslike_c': 0.6,
  'feelslike_f': 33.1,
  'windchill_c': 0.5,
  'windchill_f': 33.0,
  'heatindex_c': 4.3,
  'heatindex_f': 39.8,
  'dewpoint_c': 3.3,
  'dewpoint_f': 37.9,
  'vis_km': 9.7,
  'vis_miles': 6.0,
  'uv': 0.0,
  'gust_mph': 14.2,
  'gust_kph': 22.8}}

In [7]:


result = client.chat.completions.create(
    messages=[{"role": "developer",
               "content":"""You are an assistant who answers questions. 
               You have the following tools available. If you need to use any of these tools, your response should include the name of the tool and the input arguments to the tool.
               tools = [{tool_name: get_weather, input_arguments:{'city_name': city_name}}]

               Here are a few examples:
               user: what is the weather in Paris?
               response: {"final_answer": {}, tool_calls:{tool:get_weather, input_arguments:{city_name:Paris}}}

               Another example:
               user: What is 2 + 2 ?
               response: {"final_answer": "It is 4." , tool_calls:{}}
               """},
        {
            "role": "user",
            "content": "What is the weather in Tokyo?",
        }
    ],
    model="gpt-4o",
)

res = result.choices[0].message.content
res = eval(res)
if res['final_answer']:
    print(res['final_answer'])
else:
    if res['tool_calls']['tool']=='get_weather':
        args = res['tool_calls']['input_arguments']
        weather = get_weather(**args)
        print(weather)

{'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': 35.6895, 'lon': 139.6917, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': 1742327221, 'localtime': '2025-03-19 04:47'}, 'current': {'last_updated_epoch': 1742327100, 'last_updated': '2025-03-19 04:45', 'temp_c': 2.2, 'temp_f': 36.0, 'is_day': 0, 'condition': {'text': 'Moderate or heavy sleet', 'icon': '//cdn.weatherapi.com/weather/64x64/night/320.png', 'code': 1207}, 'wind_mph': 11.0, 'wind_kph': 17.6, 'wind_degree': 351, 'wind_dir': 'N', 'pressure_mb': 1006.0, 'pressure_in': 29.7, 'precip_mm': 3.93, 'precip_in': 0.15, 'humidity': 87, 'cloud': 100, 'feelslike_c': -2.1, 'feelslike_f': 28.2, 'windchill_c': 0.5, 'windchill_f': 33.0, 'heatindex_c': 4.3, 'heatindex_f': 39.8, 'dewpoint_c': 3.3, 'dewpoint_f': 37.9, 'vis_km': 2.4, 'vis_miles': 1.0, 'uv': 0.0, 'gust_mph': 14.2, 'gust_kph': 22.8}}


In [8]:
res

{'final_answer': {},
 'tool_calls': {'tool': 'get_weather',
  'input_arguments': {'city_name': 'Tokyo'}}}

In [9]:
# Define tool specifications
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city name"
                    }
                },
                "required": ["city"]
            }
        }
    }
]


response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "What is the weather in Tokyo?"}],
    tools=tools,
    tool_choice="auto"
)
response.choices[0].message


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_BGLgNEfq9Khldz3G297kYNad', function=Function(arguments='{"city":"Tokyo"}', name='get_weather'), type='function')])

In [5]:
tool_call = response.choices[0].message.tool_calls[0]
eval(tool_call.function.arguments)

{'city': 'Tokyo'}

In [6]:
def process_query(query):
    # Send query to OpenAI
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": query}],
        tools=tools,
        tool_choice="auto"
    )

    # Get the response message
    message = response.choices[0].message

    # Check if the model wants to call a tool
    if not message.tool_calls:
        return message.content

    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    # Convert string to dict
    arguments = eval(tool_call.function.arguments)

    # Call the appropriate function
    if function_name == "get_weather":
        function_result = get_weather(arguments["city"])
    second_response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": query},
                  message,
                  {'role':'tool',
                   'tool_call_id':tool_call.id,
                   "content":str(function_result)}],
        tools=tools,
        tool_choice="auto"
    )

    return(second_response.choices[0].message.content)


process_query("What's the weather in New York?")

"The current weather in New York is sunny with a temperature of 13.3°C (55.9°F). The wind is blowing from the north at 8.5 mph (13.7 kph). The humidity is quite low at 19%, and there's no cloud cover. The UV index is at 3.6, indicating moderate exposure risk."

## RAG

In [ ]:

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the PDF document
loader = PyPDFLoader("nasa_history.pdf")
documents = loader.load()

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    
texts = text_splitter.split_documents(documents)



Multiple definitions in dictionary at byte 0x142e for key /Length
Multiple definitions in dictionary at byte 0x3de8 for key /Length
Multiple definitions in dictionary at byte 0x750b for key /Length
Multiple definitions in dictionary at byte 0x17635 for key /Length
Multiple definitions in dictionary at byte 0x29e82 for key /Length
Multiple definitions in dictionary at byte 0x3c1db for key /Length
Multiple definitions in dictionary at byte 0x4ead0 for key /Length
Multiple definitions in dictionary at byte 0x554ba for key /Length
Multiple definitions in dictionary at byte 0x56925 for key /Length
Multiple definitions in dictionary at byte 0x6249f for key /Length
Multiple definitions in dictionary at byte 0x634e4 for key /Length
Multiple definitions in dictionary at byte 0x6a88c for key /Length
Multiple definitions in dictionary at byte 0x6bdab for key /Length
Multiple definitions in dictionary at byte 0x79cfc for key /Length
Multiple definitions in dictionary at byte 0x8b409 for key /Lengt

In [9]:
texts

[Document(metadata={'producer': 'Pdf.Capture version 5.2', 'creator': 'PyPDF', 'creationdate': 'D:00000101000000Z', 'author': '', 'title': '', 'subject': '', 'keywords': '', 'moddate': '2002-12-18T16:17:16-05:00', 'source': 'nasa_history.pdf', 'total_pages': 430, 'page': 0, 'page_label': '1'}, page_content='NASA SP-4214\n HERE NO MAN\n HAS GONE BEFORE\nA HistoryofApolloLunarExplorationMissions\nWilliam David Compton\nThe NASA History Series\n_I__A NationalAeronautics and Space AdministrationOffice of Management\nScientificand Technical Information Division\nWashington, DC t989'),
 Document(metadata={'producer': 'Pdf.Capture version 5.2', 'creator': 'PyPDF', 'creationdate': 'D:00000101000000Z', 'author': '', 'title': '', 'subject': '', 'keywords': '', 'moddate': '2002-12-18T16:17:16-05:00', 'source': 'nasa_history.pdf', 'total_pages': 430, 'page': 1, 'page_label': '2'}, page_content="Library of Congress Cataloging-in-Publication Data\nCompton, William David.\nWhere no man has gone befor

In [22]:
# Create embeddings and vector store
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(texts, embeddings)

In [24]:
from pathlib import Path
vector_store.save_local("faiss_index")

Path("faiss_index").stat().st_size
# Returns size in bytes

128

In [42]:
# Create the RAG chain
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import langchain
langchain.debug = True

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)



# Define the prompt template
prompt_template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    
    Context: {context}
    
    Question: {question}
    
    Answer: """

PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )


# Create the retrieval QA chain
qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )

In [43]:
question = "What was the main outcome of the Summer Study at Woods Hole in 1965?"
result = qa_chain({"query": question})
print(result['result'])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What was the main outcome of the Summer Study at Woods Hole in 1965?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "What was the main outcome of the Summer Study at Woods Hole in 1965?",
  "context": "Woods Hole and Falmouth Conferences, Summer 1965\nIn the summer of 1965 the Space Science Board again convened a conference\nto consider NASA's plans for space research. The Iowa summer study three years\nbefore had reviewed current programs and recommended changes of emphasis;\nthe 1965 study at Woods Hole, Massachusetts, was charged with recommending\ndirections and priorities for the next 10 to 20 years. The major topics considered\nwere lunar and planetary exploration, astronomy, and the role of humans in space\nexploration. 

In [48]:
result

ChatCompletion(id='chatcmpl-BAkP67kqI0VgZ5vPGRvWhVF5yxYnK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"final_answer": {}, "tool_calls": {"tool": "get_weather", "input_arguments": {"city_name": "Tokyo"}}}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1741901356, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_eb9dce56a8', usage=CompletionUsage(completion_tokens=29, prompt_tokens=158, total_tokens=187, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))